In [1]:
import jax
import flax
import hard_not
import harden

In [10]:
weights

FrozenDict({
    params: {
        weights: DeviceArray([[0.26377404, 0.8707025 ],
                     [0.44444847, 0.03216302],
                     [0.6110164 , 0.685097  ],
                     [0.9133855 , 0.08662593]], dtype=float32),
    },
})

In [2]:
dict = flax.core.frozen_dict.FrozenDict({'a': 0.5, 'b': 0.6, 'c': 0.4, 'd': 0.0, 'e': 1.0})
expected_dict = {'a': False, 'b': True, 'c': False, 'd': False, 'e': True}
print(dict)

FrozenDict({
    a: 0.5,
    b: 0.6,
    c: 0.4,
    d: 0.0,
    e: 1.0,
})


In [3]:
harden.harden_dict(dict)

{'a': False, 'b': True, 'c': False, 'd': False, 'e': True}

In [4]:
from unittest import TestCase

In [7]:
import json

In [8]:
dict = {'a': 0.5, 'b': 0.6, 'c': 0.4, 'd': 0.0, 'e': 1.0, 'f': jax.numpy.array([0.5, 0.6, 0.4, 0.0, 1.0])}
expected_dict = {'a': False, 'b': True, 'c': False, 'd': False, 'e': True, 'f': jax.numpy.array([False, True, False, False, True])}
str(expected_dict) == str(harden.harden(dict))

TypeError: Object of type DeviceArray is not JSON serializable